<a href="https://colab.research.google.com/github/Abdelkader-Kaddour-Brahim/Abdelkader-Kaddour-Brahim/blob/main/RAG-LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain_community langchain-huggingface langchain -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.0 MB/s eta 0:00:00


In [2]:
pip install faiss-cpu -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 65.7 MB/s eta 0:00:00


In [5]:
import faiss
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_huggingface import HuggingFaceEndpoint
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
import numpy as np
import os

In [ ]:
## for token use www.huggingface.co and create new token
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "your_hf_token_here"

In [ ]:
with open('YOUR_TXT_FILE.txt','r',encoding='utf-8') as file_reader:
  text = file_reader.read()
  file_reader.close()


In [ ]:
text_splitter = CharacterTextSplitter(chunk_size = 100, chunk_overlap= 200)
chunks = text_splitter.split_text(text)

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks = text_splitter.split_text(text)

In [ ]:
hf_embeddings = HuggingFaceEmbeddings(
    model_name = "sentence-transformers/all-mpnet-base-v2",
    model_kwargs = {'device':'cuda'},
    encode_kwargs = {'normalize_embeddings':False
                     })

In [ ]:
embeddings = hf_embeddings.embed_documents(chunks)
dimension = len(embeddings[0])

In [ ]:
quantizer = faiss.IndexFlatL2(dimension)
index = faiss.IndexIVFFlat(quantizer, dimension, 100)
index.train(np.array(embeddings).astype('float32'))
index.add(np.array(embeddings).astype('float32'))

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm,retriever=vector_search.as_retriever())
answer = qa({'query':query})
print(answer)

In [ ]:
documents = [Document(page_content=chunk) for chunk in chunks]
docstore = InMemoryDocstore({str(i): doc for i, doc in enumerate(documents)})
index_to_docstore_id = {i: str(i) for i in range(len(documents))}

vector_store = FAISS(embedding_function=hf_embeddings.embed_query,index=index, docstore=docstore, index_to_docstore_id=index_to_docstore_id)

vector_store.save_local('vector_search')

In [ ]:
llm = HuggingFaceEndpoint(
    repo_id = "mistralai/Mistral-7B-Instruct-v0.2",
    task = "text-generation",
    temperature=0.6
)

In [ ]:
vector_search = FAISS.load_local("/content/vector_search/",embeddings=hf_embeddings, allow_dangerous_deserialization=True)
query = "tap something"

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm,retriever=vector_search.as_retriever())
answer = qa({'query':query})
print(answer)